In [5]:
import pandas as pd

In [141]:
#read from flat excel waste per category
df1_waste_cate = pd.read_excel('../source/waste_cate.xlsx', 'Sheet 1') 
df2_waste_cate = pd.read_excel('../source/waste_cate.xlsx', 'Sheet 2')
#read from flat excel food waste 
df1_food_waste = pd.read_excel('../source/food_waste.xlsx','Sheet 1')
df2_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 2')
df3_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 3')
df4_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 4')

#read from text file 
df_composition = pd.read_csv('../source/food_composition.csv')

#read from bigquery csv
df_bq_keywords = pd.read_csv('../source/bq_keywords_food_waste.csv')

#read from API csv
df_api_12 = pd.read_csv('../source/api_data12.csv')
df_api_12_3 = pd.read_csv('../source/api_data12_3.csv')

df_top_fruit_veg_wasted = pd.read_csv('../source/top_fruit_veg_wasted.csv')




c:\Users\krant\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\krant\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\krant\AppData\Local\Temp\ipykernel_26224\510886270.py:14: DtypeWarning: Columns (9,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bq_keywords = pd.read_csv('../source/bq_keywords_food_waste.csv')


In [140]:
#read from flat excel waste per category
 

# Slice rows from index 8 to 48 of df1_waste_cate and reset the index, dropping the old index
df_waste = df1_waste_cate[8:49].reset_index(drop=True)

# Rename the columns of df_waste using the values from the 9th row of df1_waste_cate
df_waste = df_waste.rename(columns=df1_waste_cate.iloc[7])

# Add a new column named 'year' to df_waste and set all values in that column to 2020
df_waste['year'] = 2020

# Slice rows from index 8 to 48 of df2_waste_cate and reset the index, dropping the old index
df_waste2 = df2_waste_cate[8:49].reset_index(drop=True)

# Rename the columns of df_waste2 using the values from the 9th row of df2_waste_cate
df_waste2 = df_waste2.rename(columns=df2_waste_cate.iloc[7])

# Add a new column named 'year' to df_waste2 and set all values in that column to 2021
df_waste2['year'] = 2021

# Concatenate df_waste and df_waste2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_waste = pd.concat([df_waste, df_waste2]).reset_index(drop=True)


df_waste

,COICOP (Labels),Total,NaN,Food and non-alcoholic beverages,NaN,Food,NaN,Non-alcoholic beverages,NaN,"Alcoholic beverages, tobacco and narcotics",...,NaN,Social protection,NaN,Insurance,NaN,Financial services n.e.c.,NaN,Other services n.e.c.,NaN,year
0,GEO (Labels),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
1,European Union - 27 countries (from 2020),100,NaN,14.8,NaN,13.4,NaN,1.4,NaN,4.5,...,NaN,86944.5,NaN,184448.1,NaN,168105.4,NaN,:,NaN,2020
2,Euro area - 19 countries (2015-2022),100,NaN,14.3,NaN,13,NaN,1.3,NaN,4.3,...,NaN,77863,NaN,170124.5,NaN,139379.6,NaN,:,NaN,2020
3,Belgium,100,NaN,14.2,NaN,13.1,NaN,1.1,NaN,4.4,...,NaN,2510.9,NaN,6691.3,NaN,7746.5,NaN,4539.9,NaN,2020
4,Bulgaria,100,NaN,20.7,NaN,19,NaN,1.7,NaN,5.7,...,NaN,4.6,NaN,255.7,NaN,236.1,NaN,392.6,NaN,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,North Macedonia,100,NaN,30.5,NaN,27.2,NaN,3.4,NaN,5.2,...,NaN,:,NaN,:,NaN,:,NaN,:,NaN,2021
78,Albania,:,NaN,:,NaN,:,NaN,:,NaN,:,...,NaN,:,NaN,:,NaN,:,NaN,:,NaN,2021
79,Serbia,100,NaN,23.2,NaN,20.6,NaN,2.6,NaN,8.5,...,NaN,77.9,NaN,255.7,NaN,893.2,NaN,239.3,NaN,2021
80,Türkiye,100,NaN,23.3,NaN,:,c,:,c,3,...,NaN,:,NaN,:,NaN,:,NaN,:,NaN,2021


In [99]:
# Slice rows from index 10 to 38 of df1_food_waste and reset the index, dropping the old index
df_kilo_capita = df1_food_waste[10:39].reset_index(drop=True)

# Rename the columns of df_kilo_capita using the values from the 9th row of df1_food_waste
df_kilo_capita = df_kilo_capita.rename(columns=df1_food_waste.iloc[9])

# Add a new column named 'year' to df_kilo_capita and set all values in that column to 2020
df_kilo_capita['year'] = 2020

# Slice rows from index 10 to 38 of df2_food_waste and reset the index, dropping the old index
df_kilo_capita2 = df2_food_waste[10:39].reset_index(drop=True)

# Rename the columns of df_kilo_capita2 using the values from the 9th row of df2_food_waste
df_kilo_capita2 = df_kilo_capita2.rename(columns=df2_food_waste.iloc[9])

# Add a new column named 'year' to df_kilo_capita2 and set all values in that column to 2021
df_kilo_capita2['year'] = 2021

# Concatenate df_kilo_capita and df_kilo_capita2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_kilo_capita = pd.concat([df_kilo_capita, df_kilo_capita2]).reset_index(drop=True)



In [100]:
# Slice rows from index 10 to 38 of df3_food_waste and reset the index, dropping the old index
df_tonne = df3_food_waste[10:39].reset_index(drop=True)

# Rename the columns of df_tonne using the values from the 9th row of df3_food_waste
df_tonne = df_tonne.rename(columns=df3_food_waste.iloc[9])

# Add a new column named 'year' to df_tonne and set all values in that column to 2020
df_tonne['year'] = 2020

# Slice rows from index 10 to 38 of df4_food_waste and reset the index, dropping the old index
df_tonne2 = df4_food_waste[10:39].reset_index(drop=True)

# Rename the columns of df_tonne2 using the values from the 9th row of df4_food_waste
df_tonne2 = df_tonne2.rename(columns=df4_food_waste.iloc[9])

# Add a new column named 'year' to df_tonne2 and set all values in that column to 2021
df_tonne2['year'] = 2021

# Concatenate df_tonne and df_tonne2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_tonne = pd.concat([df_tonne, df_tonne2]).reset_index(drop=True)



In [133]:
df_composition


,Category,Denmark (Foodwasteatprimaryproduction(kt)),Germany (Foodwasteatprimaryproduction(kt)),Italy (Foodwasteatprimaryproduction(kt)),Denmark (Foodwasteatmanufacturing(kt)),Germany (Foodwasteatmanufacturing(kt)),Italy (Foodwasteatmanufacturing(kt))
0,Meat,0.00,0.00,0.00,20.13,186.69,143.55
1,Fish,136.50,58.06,66.46,27.89,80.37,37.75
2,Dairy,14.90,89.16,33.45,43.63,218.16,139.81
3,Eggs,1.41,14.42,14.76,1.35,16.90,13.59
4,Cereals,2.06,45.06,18.99,16.42,212.97,44.71
5,Fruits,2.02,99.25,811.30,7.81,135.46,679.84
6,Vegetables,26.61,332.97,868.17,24.05,270.29,570.32
7,Potatoes,0.00,0.00,0.00,9.45,192.46,10.97
8,Sugarbeets,0.00,0.00,0.00,0.00,0.00,0.00
9,Oilcrops,1.24,15.18,0.69,0.00,0.01,59.23


In [139]:
df_bq_keywords

,goal,target,indicator,seriescode,seriesdescription,geoareacode,geoareaname,timeperiod,value,time_detail,...,name_of_international_institution,name_of_non_communicable_disease,sex,tariff_regime_status,type_of_mobile_technology,type_of_occupation,type_of_product,type_of_skill,type_of_speed,units
0,3,3.5,3.5.2,SH_ALC_CONSPT,Alcohol consumption per capita (aged 15 years ...,662,Saint Lucia,2016,9.85179,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LTPUREAL
1,3,3.5,3.5.2,SH_ALC_CONSPT,Alcohol consumption per capita (aged 15 years ...,328,Guyana,2000,8.62865,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LTPUREAL
2,3,3.5,3.5.2,SH_ALC_CONSPT,Alcohol consumption per capita (aged 15 years ...,862,Venezuela (Bolivarian Republic of),2005,9.34646,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LTPUREAL
3,3,3.5,3.5.2,SH_ALC_CONSPT,Alcohol consumption per capita (aged 15 years ...,598,Papua New Guinea,2015,1.26335,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LTPUREAL
4,3,3.5,3.5.2,SH_ALC_CONSPT,Alcohol consumption per capita (aged 15 years ...,4,Afghanistan,2016,0.22081,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LTPUREAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350935,2,2.2,2.2.2,SH_STA_WASTE,Proportion of children moderately or severely ...,454,Malawi,2015,2.70000,2015-16 (2015),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PER_POP_U5
350936,2,2.2,2.2.2,SH_STA_WASTE,Proportion of children moderately or severely ...,84,Belize,2015,1.80000,2015-16 (2015),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PER_POP_U5
350937,2,2.2,2.2.2,SH_STA_WASTE,Proportion of children moderately or severely ...,834,United Republic of Tanzania,2015,4.50000,2015-16 (2015),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PER_POP_U5
350938,2,2.2,2.2.2,SH_STA_WASTE,Proportion of children moderately or severely ...,108,Burundi,2016,5.10000,2016-17 (2016),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PER_POP_U5


In [136]:
df_api_12

,goal,target,indicator,series,seriesDescription,seriesCount,geoAreaCode,geoAreaName,timePeriodStart,value,...,time_detail,timeCoverage,upperBound,lowerBound,basePeriod,source,geoInfoUrl,footnotes,attributes,dimensions
0,"['12', '8']","['12.2', '8.4']","['12.2.1', '8.4.1']",EN_MAT_FTPRPG,"Material footprint per unit of GDP, by type of...",23,1,World,2000.0,1.188740e+00,...,NaN,NaN,NaN,NaN,2015.0,WESR / Global Material Flows Database,NaN,[''],"{'Nature': 'N', 'Units': 'KG_PER_CON_USD', 'Ob...","{'Type of product': 'ALP', 'Reporting Type': 'G'}"
1,"['12', '8']","['12.2', '8.4']","['12.2.1', '8.4.1']",EN_MAT_FTPRPC,"Material footprint per capita, by type of raw ...",23,1,World,2000.0,9.335630e+00,...,NaN,NaN,NaN,NaN,NaN,WESR / Global Material Flows Database,NaN,[''],"{'Nature': 'N', 'Units': 'TONNES', 'Observatio...","{'Type of product': 'ALP', 'Reporting Type': 'G'}"
2,"['12', '8']","['12.2', '8.4']","['12.2.1', '8.4.1']",EN_MAT_FTPRTN,"Material footprint, by type of raw material (t...",23,1,World,2000.0,5.740385e+10,...,NaN,NaN,NaN,NaN,NaN,WESR / Global Material Flows Database,NaN,[''],"{'Nature': 'N', 'Units': 'TONNES', 'Observatio...","{'Type of product': 'ALP', 'Reporting Type': 'G'}"
3,"['12', '8']","['12.2', '8.4']","['12.2.2', '8.4.2']",EN_MAT_DOMCMPT,"Domestic material consumption, by type of raw ...",94414,4,Afghanistan,2000.0,5.475248e+06,...,NaN,NaN,NaN,NaN,NaN,WESR / Global Material Flows Database,NaN,[''],"{'Nature': 'E', 'Units': 'TONNES', 'Observatio...","{'Type of product': 'CRO', 'Reporting Type': 'G'}"
4,"['12', '8']","['12.2', '8.4']","['12.2.2', '8.4.2']",EN_MAT_DOMCMPT,"Domestic material consumption, by type of raw ...",94414,4,Afghanistan,2000.0,2.335034e+07,...,NaN,NaN,NaN,NaN,NaN,WESR / Global Material Flows Database,NaN,[''],"{'Nature': 'E', 'Units': 'TONNES', 'Observatio...","{'Type of product': 'BIM', 'Reporting Type': 'G'}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14545,"['7', '12']","['7.b', '12.a']","['7.b.1', '12.a.1']",EG_EGY_RNEW,Installed renewable electricity-generating cap...,41216,716,Zimbabwe,2000.0,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,Renewable electricity generating capacity from...,NaN,['Year-end value for installed capacity (watts...,"{'Nature': 'E', 'Units': 'W_PER_CAPITA'}","{'Reporting Type': 'G', 'Type of renewable tec..."
14546,"['7', '12']","['7.b', '12.a']","['7.b.1', '12.a.1']",EG_EGY_RNEW,Installed renewable electricity-generating cap...,41216,716,Zimbabwe,2000.0,5.637543e+01,...,NaN,NaN,NaN,NaN,NaN,Renewable electricity generating capacity from...,NaN,['Year-end value for installed capacity (watts...,"{'Nature': 'E', 'Units': 'W_PER_CAPITA'}","{'Reporting Type': 'G', 'Type of renewable tec..."
14547,"['7', '12']","['7.b', '12.a']","['7.b.1', '12.a.1']",EG_EGY_RNEW,Installed renewable electricity-generating cap...,41216,716,Zimbabwe,2000.0,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,Renewable electricity generating capacity from...,NaN,['Year-end value for installed capacity (watts...,"{'Nature': 'E', 'Units': 'W_PER_CAPITA'}","{'Reporting Type': 'G', 'Type of renewable tec..."
14548,"['7', '12']","['7.b', '12.a']","['7.b.1', '12.a.1']",EG_EGY_RNEW,Installed renewable electricity-generating cap...,41216,716,Zimbabwe,2000.0,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,Renewable electricity generating capacity from...,NaN,['Year-end value for installed capacity (watts...,"{'Nature': 'E', 'Units': 'W_PER_CAPITA'}","{'Reporting Type': 'G', 'Type of renewable tec..."


In [137]:
df_api_12_3

,goal,target,indicator,series,seriesDescription,seriesCount,geoAreaCode,geoAreaName,timePeriodStart,value,...,time_detail,timeCoverage,upperBound,lowerBound,basePeriod,source,geoInfoUrl,footnotes,attributes,dimensions
0,['12'],['12.3'],['12.3.1'],AG_FOOD_WST_PC,Food waste per capita (KG),2171,4,Afghanistan,2019.0,125.91953,...,NaN,NaN,NaN,NaN,NaN,Food Waste Index Report / WESR,NaN,[''],"{'Nature': 'E', 'Units': 'KG', 'Observation St...","{'Reporting Type': 'G', 'Food Waste Sector': '..."
1,['12'],['12.3'],['12.3.1'],AG_FOOD_WST_PC,Food waste per capita (KG),2171,4,Afghanistan,2019.0,82.31914,...,NaN,NaN,NaN,NaN,NaN,Food Waste Index Report / WESR,NaN,[''],"{'Nature': 'E', 'Units': 'KG', 'Observation St...","{'Reporting Type': 'G', 'Food Waste Sector': '..."
2,['12'],['12.3'],['12.3.1'],AG_FOOD_WST_PC,Food waste per capita (KG),2171,4,Afghanistan,2019.0,15.75299,...,NaN,NaN,NaN,NaN,NaN,Food Waste Index Report / WESR,NaN,[''],"{'Nature': 'E', 'Units': 'KG', 'Observation St...","{'Reporting Type': 'G', 'Food Waste Sector': '..."
3,['12'],['12.3'],['12.3.1'],AG_FOOD_WST_PC,Food waste per capita (KG),2171,4,Afghanistan,2019.0,27.84740,...,NaN,NaN,NaN,NaN,NaN,Food Waste Index Report / WESR,NaN,[''],"{'Nature': 'E', 'Units': 'KG', 'Observation St...","{'Reporting Type': 'G', 'Food Waste Sector': '..."
4,['12'],['12.3'],['12.3.1'],AG_FOOD_WST_PC,Food waste per capita (KG),2171,4,Afghanistan,2022.0,32.48636,...,NaN,NaN,NaN,NaN,NaN,Food Waste Index Report / WESR,NaN,['Very Low Confidence'],"{'Nature': 'E', 'Units': 'KG', 'Observation St...","{'Reporting Type': 'G', 'Food Waste Sector': '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4739,['12'],['12.3'],['12.3.1'],AG_FLS_PCT,Food loss percentage (%),105,155,Western Europe,2020.0,6.50000,...,NaN,NaN,NaN,NaN,NaN,FAO,NaN,['Prior to July 2022: data on “Food loss perce...,"{'Nature': 'N', 'Units': 'PERCENT', 'Observati...",{'Reporting Type': 'G'}
4740,['12'],['12.3'],['12.3.1'],AG_FLS_PCT,Food loss percentage (%),105,155,Western Europe,2021.0,7.84000,...,NaN,NaN,NaN,NaN,NaN,FAO,NaN,['Prior to July 2022: data on “Food loss perce...,"{'Nature': 'N', 'Units': 'PERCENT', 'Observati...",{'Reporting Type': 'G'}
4741,['12'],['12.3'],['12.3.1'],AG_FLS_PCT,Food loss percentage (%),105,1,World,2016.0,13.00000,...,NaN,NaN,NaN,NaN,NaN,FAO,NaN,['Prior to July 2022: data on “Food loss perce...,"{'Nature': 'N', 'Units': 'PERCENT', 'Observati...",{'Reporting Type': 'G'}
4742,['12'],['12.3'],['12.3.1'],AG_FLS_PCT,Food loss percentage (%),105,1,World,2020.0,13.30000,...,NaN,NaN,NaN,NaN,NaN,FAO,NaN,['Prior to July 2022: data on “Food loss perce...,"{'Nature': 'N', 'Units': 'PERCENT', 'Observati...",{'Reporting Type': 'G'}


In [142]:
df_top_fruit_veg_wasted

,top_fruit_veg_wasted
0,Salad leaves (bagged)
1,Bananas
2,Lettuce (whole)
3,Cucumber
4,Tomatoes
5,Carrots
6,Mushrooms
7,Potato
8,Grapes
9,Strawberries
